In [1]:
import pandas as pd
from IPython.display import display

In [62]:
from IPython.display import display
import pandas as pd
def analyze_results(directory, log_file, output_file, print_flag=False):
    # Concatenate the directory with the filenames
    log_file = directory+log_file
    output_file = directory+output_file    
    # Get the data from the formatted csv
    data = pd.read_csv(log_file, sep=' ')
    # Rename runtime to Runtime (seconds)
    data = data.rename(columns={'runtime':'Runtime (seconds)'})
    # Create the ExcelWriter to send the information to xlsx
    writer = pd.ExcelWriter(output_file)
    
    # Loop over the choices for ending time (tFinal)
    for time in data.tFinal.unique():
        # Loop over the choices for the Reynolds number (Re)
        for Re in data.Re.unique():
            # Get the information for a single pair of time and Re
            df = data[data.tFinal.eq(time) & data.Re.eq(Re)].reset_index(drop=True)
            # Get the reduction in error from the increase  in spatial/temporal resolution
            df['uReduction'] = df['uErr'].shift(1)/df['uErr']
            df['pReduction'] = df['pErr'].shift(1)/df['pErr']
            # Write results to a sheet in output_file
            sheetname = 'tFinal='+str(time)+', Re='+str(Re)
            df.to_excel(writer,
                       sheet_name=sheetname,
                       index=False,
                       columns=['Re','nx','nt',
                                    'uErr','uReduction',
                                   'pErr','pReduction',
                                   'Runtime (seconds)'],)
            workbook = writer.book
            format_all = workbook.add_format()
            format_all.set_align('center')
            sheet = writer.sheets[sheetname]
            width = 18
            sheet.set_column('A:I',width,format_all)
            # Specify uReduction, pReduction, and Runtime (seconds) to having only 2 decimal places
            format_decimals = workbook.add_format()
            format_decimals.set_align('center')
            format_decimals.set_num_format('0.00')            
            sheet.set_column('E:E',width,format_decimals)
            sheet.set_column('G:G',width,format_decimals)
            sheet.set_column('H:H',width,format_decimals)
            if print_flag:
                display(df)
    
    writer.save()
    writer.close()

In [69]:
import pandas as pd
from IPython.display import display
def analyze_rom_results(directory, log_file, output_file, print_flag=False):
    """
    Purpose:
    To take data from a bash script run of the Inverse Problem that has been grepped and format the data
    before saving it into sheets in an Excel file grouped by tFinal and Re
    
    Keyword Arugments:
    directory - the directory the log_file is stored in and where the Excel file will be saved
    log_file - name of the grepped text file containing the results of a series of runs for the Inverse Problem
    output_file - name of the Excel file to be saved
    print_flag - boolean flag used to turn on print statements of the resulting DataFrames (default is False)
    
    Returns:
    None
    """
    # Concatenate the directory with the filenames
    log_file = directory+log_file
    output_file = directory+output_file    
    # Get the data from the formatted csv
    data = pd.read_csv(log_file, sep=' ')
    # Rename runtime to Runtime (seconds)
    data = data.rename(columns={'runtime':'Runtime (seconds)'})
    # Create the ExcelWriter to send the information to xlsx
    writer = pd.ExcelWriter(output_file)
    
    # Loop over the choices for ending time (tFinal)
    for time in data.tFinal.unique():
        # Loop over the choices for the Reynolds number (Re)
        for Re in data.Re.unique():
            # Get the information for a single pair of time and Re
            df = data[data.tFinal.eq(time) & data.Re.eq(Re)].reset_index(drop=True)
            # Get the reduction in error from the increase  in spatial/temporal resolution
            df['alphaReduction'] = df['alphaErr2'].shift(1)/df['alphaErr2']
            df['uReduction'] = df['uErr2'].shift(1)/df['uErr2']
            # Write results to a sheet in output_file
            sheetname = 'tFinal='+str(time)+', Re='+str(Re)
            df.to_excel(writer,
                       sheet_name=sheetname,
                       index=False,
                       columns=['Re','nx','nSteps',
                                    'uErr2','uReduction',
                                   'alphaErr2','alphaReduction',
                                   'InverseTime','ErrorCheckRunTime','VTKRunTime','Total'])
            workbook = writer.book
            # Base format used for all columns
            format_all = workbook.add_format()
            format_all.set_align('center')
            # Format the sheet
            sheet = writer.sheets[sheetname]
            width = 18
            sheet.set_column('A:I',width,format_all)
            # Format uReduction, alphaReduction
            format_reduction = workbook.add_format()
            format_reduction.set_align('center')
            format_reduction.set_num_format('0.00')            
            sheet.set_column('E:E',width,format_reduction)
            sheet.set_column('G:G',width,format_reduction)
            # Format the runtime columns
            format_time = workbook.add_format()
            format_time.set_num_format('0.00')
            sheet.set_column('H:K',width,format_time)
            if print_flag:
                display(df)
    
    writer.save()
    writer.close()

In [63]:
# This block is for the Viento Verification
FP_dir = '/home/sirush/PhDResearch/MMS_Transient_Channel/ForwardProblem/Log_Files/'
viento_verification_log = 'log_Viento_verification_short.txt'
viento_verification_output = 'Viento_Verification.xlsx'
analyze_results(directory=FP_dir,log_file=viento_verification_log,output_file=viento_verification_output)

In [68]:
# This block is for getting the Inverse Problem results for comparing with the ODECO single parameter space
InvProb_dir = '/home/sirush/PhDResearch/MMS_Transient_Channel/InverseProblem/Log_Files/'
inv_prob_log = 'log_short_Re{1,20,40,60,80,100}_nxnt{24,36,48,60,72}.txt'
inv_prob_output = 'Inverse_Problem_Results.xlsx'
analyze_rom_results(directory=InvProb_dir,log_file=inv_prob_log,output_file=inv_prob_output)


